In [1]:
import numpy as np
import pandas as pd
import os
from PIL import Image
import warnings
from sklearn.model_selection import train_test_split

In [2]:
# Get all the images with their classes from the vehicles dataset
def get_images_and_classes(data_dir):
    images = []
    classes = []
    for class_dir in os.listdir(data_dir):
        class_path = os.path.join(data_dir, class_dir)
        if os.path.isdir(class_path):
            for img_file in os.listdir(class_path):
                if img_file.endswith(('.png', '.jpg', '.jpeg')):
                    images.append(os.path.join(class_path, img_file))
                    classes.append(class_dir)
    return images, classes

data_directory = 'Vehicles'
images, classes = get_images_and_classes(data_directory)

In [3]:
# Preprocess the images
def preprocess_images(image_paths, target_size=(128, 128)):

    processed_images = []
    for img_path in image_paths:
        img = Image.open(img_path).convert('RGB')
        img = img.resize(target_size)
        img_array = np.array(img) / 255.0  # Normalize to [0, 1]
        processed_images.append(img_array)
    return np.array(processed_images)

processed_images = preprocess_images(images)

c:\Users\Shashank Goel\AppData\Local\Programs\Python\Python313\Lib\site-packages\PIL\Image.py:1039: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


In [4]:
processed_images.shape

(5504, 128, 128, 3)

In [5]:
# CNN
import tensorflow as tf
from tensorflow.keras import layers, models

def create_cnn_model(input_shape, num_classes):
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(num_classes, activation='softmax'))
    
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

input_shape = (128, 128, 3)
num_classes = len(set(classes))
model = create_cnn_model(input_shape, num_classes)
model.summary()

c:\Users\Shashank Goel\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 7)              │           903 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,305,543 (12.61 MB)

 Trainable params: 3,305,543 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:
# Create a mapping from class names to numeric labels
unique_classes = sorted(set(classes))
class_to_label = {class_name: idx for idx, class_name in enumerate(unique_classes)}

# Convert classes to numeric labels
y = np.array([class_to_label[c] for c in classes])

# Now split the data
X_train, X_test, y_train, y_test = train_test_split(processed_images, y, test_size=0.2, random_state=42)

In [8]:
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
111/111 ━━━━━━━━━━━━━━━━━━━━ 14s 118ms/step - accuracy: 0.4532 - loss: 1.4478 - val_accuracy: 0.6129 - val_loss: 1.1466
Epoch 2/10
111/111 ━━━━━━━━━━━━━━━━━━━━ 12s 111ms/step - accuracy: 0.7209 - loss: 0.8237 - val_accuracy: 0.7117 - val_loss: 0.8615
Epoch 3/10
111/111 ━━━━━━━━━━━━━━━━━━━━ 12s 111ms/step - accuracy: 0.7927 - loss: 0.6250 - val_accuracy: 0.7628 - val_loss: 0.6902
Epoch 4/10
111/111 ━━━━━━━━━━━━━━━━━━━━ 12s 110ms/step - accuracy: 0.8489 - loss: 0.4396 - val_accuracy: 0.7832 - val_loss: 0.7005
Epoch 5/10
111/111 ━━━━━━━━━━━━━━━━━━━━ 12s 111ms/step - accuracy: 0.8981 - loss: 0.3025 - val_accuracy: 0.7821 - val_loss: 0.6927
Epoch 6/10
111/111 ━━━━━━━━━━━━━━━━━━━━ 12s 110ms/step - accuracy: 0.9338 - loss: 0.2016 - val_accuracy: 0.8025 - val_loss: 0.6696
Epoch 7/10
111/111 ━━━━━━━━━━━━━━━━━━━━ 12s 110ms/step - accuracy: 0.9654 - loss: 0.1089 - val_accuracy: 0.7980 - val_loss: 0.7674
Epoch 8/10
111/111 ━━━━━━━━━━━━━━━━━━━━ 12s 112ms/step - accuracy: 0.9767 - loss: 0

In [9]:
loss, accuracy = model.evaluate(X_train, y_train)
print(f'Training accuracy: {accuracy:.4f}')
print(f'Training loss: {loss:.4f}')

loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test accuracy: {accuracy:.4f}')
print(f'Test loss: {loss:.4f}')

138/138 ━━━━━━━━━━━━━━━━━━━━ 5s 33ms/step - accuracy: 0.9614 - loss: 0.2116
Training accuracy: 0.9614
Training loss: 0.2116
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - accuracy: 0.8029 - loss: 0.9980
Test accuracy: 0.8029
Test loss: 0.9980


In [10]:
# Check for overfitting by comparing training vs test performance
loss_train, accuracy_train = model.evaluate(X_train, y_train, verbose=0)
loss_test, accuracy_test = model.evaluate(X_test, y_test, verbose=0)

print(f'Training accuracy: {accuracy_train:.4f}')
print(f'Test accuracy: {accuracy_test:.4f}')
print(f'Accuracy difference: {accuracy_train - accuracy_test:.4f}')

print(f'\nTraining loss: {loss_train:.4f}')
print(f'Test loss: {loss_test:.4f}')
print(f'Loss difference: {loss_test - loss_train:.4f}')

Training accuracy: 0.9614
Test accuracy: 0.8029
Accuracy difference: 0.1585

Training loss: 0.2116
Test loss: 0.9980
Loss difference: 0.7865


In [11]:
model.save('vehicle_classification_model.keras')